# Risk Modelling & Portfolio Optimization in Python
**Week 5 - Financial ML Bootcamp**

## Overview
This notebook demonstrates risk modelling and portfolio optimization techniques using:
- Modern Portfolio Theory (MPT)
- Monte Carlo simulation for portfolio analysis
- Value at Risk (VaR) and Conditional Value at Risk (CVaR) calculation
- Efficient Frontier visualization
- Sharpe ratio optimization

## Learning Objectives
- Quantify portfolio risk using statistical measures
- Implement Monte Carlo simulation for portfolio analysis
- Optimize portfolio weights for maximum Sharpe ratio
- Visualize the efficient frontier
- Calculate VaR and CVaR for risk management

---

In [ ]:
# Parameters and Configuration
SEED = 42
SAMPLE_MODE = True  # Set to False for full analysis
DATA_PATH = "data/portfolio_assets.csv"

# Portfolio configuration
ASSETS = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'META']
START_DATE = '2019-01-01'
END_DATE = '2024-01-01'
NUM_PORTFOLIOS = 10000 if not SAMPLE_MODE else 1000
CONFIDENCE_LEVEL = 0.95
RISK_FREE_RATE = 0.02  # 2% annual risk-free rate

print(f"Configuration:")
print(f"- Sample Mode: {SAMPLE_MODE}")
print(f"- Assets: {ASSETS}")
print(f"- Date Range: {START_DATE} to {END_DATE}")
print(f"- Number of Portfolios: {NUM_PORTFOLIOS}")
print(f"- Confidence Level: {CONFIDENCE_LEVEL}")

In [ ]:
# Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from scipy.optimize import minimize
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(SEED)

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ Libraries imported successfully")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## Step 1: Data Loading and Preprocessing

We'll load historical price data for a diversified portfolio of technology stocks and compute daily returns.

In [ ]:
def load_portfolio_data(assets, start_date, end_date, sample_mode=True):
    """
    Load historical price data for portfolio assets.
    
    Parameters:
    - assets: List of ticker symbols
    - start_date: Start date for data
    - end_date: End date for data
    - sample_mode: If True, use fallback synthetic data if download fails
    
    Returns:
    - prices_df: DataFrame with adjusted closing prices
    - returns_df: DataFrame with daily returns
    """
    try:
        print(f"📊 Loading data for {len(assets)} assets...")
        
        # Download data from Yahoo Finance
        data = yf.download(assets, start=start_date, end=end_date, progress=False)
        prices_df = data['Adj Close'].dropna()
        
        # Ensure we have data for all assets
        if prices_df.empty or len(prices_df.columns) < len(assets):
            raise ValueError("Incomplete data downloaded")
            
        print(f"✅ Successfully loaded {len(prices_df)} days of data")
        
    except Exception as e:
        print(f"⚠️ Data download failed: {e}")
        if sample_mode:
            print("🔄 Generating synthetic data for demonstration...")
            prices_df = generate_synthetic_data(assets, start_date, end_date)
        else:
            raise
    
    # Calculate daily returns
    returns_df = prices_df.pct_change().dropna()
    
    # Display basic statistics
    print(f"\nData Summary:")
    print(f"- Date range: {prices_df.index[0].strftime('%Y-%m-%d')} to {prices_df.index[-1].strftime('%Y-%m-%d')}")
    print(f"- Number of trading days: {len(returns_df)}")
    print(f"- Assets: {list(prices_df.columns)}")
    
    return prices_df, returns_df

def generate_synthetic_data(assets, start_date, end_date):
    """Generate synthetic price data for demonstration purposes."""
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    n_days = len(dates)
    
    # Synthetic parameters (realistic for tech stocks)
    np.random.seed(SEED)
    initial_prices = np.random.uniform(100, 300, len(assets))
    annual_returns = np.random.uniform(0.05, 0.25, len(assets))
    volatilities = np.random.uniform(0.15, 0.35, len(assets))
    
    # Generate correlated returns
    correlation_matrix = np.random.uniform(0.3, 0.7, (len(assets), len(assets)))
    correlation_matrix = (correlation_matrix + correlation_matrix.T) / 2
    np.fill_diagonal(correlation_matrix, 1.0)
    
    # Generate returns using multivariate normal distribution
    daily_returns = np.random.multivariate_normal(
        annual_returns / 252,  # Convert to daily
        np.outer(volatilities, volatilities) * correlation_matrix / 252,
        n_days
    )
    
    # Convert to prices
    prices = np.zeros((n_days, len(assets)))
    prices[0] = initial_prices
    
    for i in range(1, n_days):
        prices[i] = prices[i-1] * (1 + daily_returns[i])
    
    return pd.DataFrame(prices, index=dates, columns=assets)

# Load the data
prices_df, returns_df = load_portfolio_data(ASSETS, START_DATE, END_DATE, SAMPLE_MODE)

# Display first few rows
print(f"\nReturns Data (first 5 rows):")
print(returns_df.head())

## Step 2: Calculate Portfolio Statistics

We'll compute the expected returns, volatilities, and covariance matrix needed for portfolio optimization.

In [ ]:
def calculate_portfolio_stats(returns_df):
    """
    Calculate portfolio statistics from returns data.
    
    Parameters:
    - returns_df: DataFrame with daily returns
    
    Returns:
    - expected_returns: Mean annual returns
    - cov_matrix: Annual covariance matrix
    - correlation_matrix: Correlation matrix
    """
    # Annualize returns and covariance (252 trading days per year)
    expected_returns = returns_df.mean() * 252
    cov_matrix = returns_df.cov() * 252
    correlation_matrix = returns_df.corr()
    
    return expected_returns, cov_matrix, correlation_matrix

def display_portfolio_stats(expected_returns, cov_matrix, correlation_matrix):
    """Display portfolio statistics in a formatted way."""
    print("📈 Portfolio Statistics Summary")
    print("=" * 50)
    
    # Expected returns
    print("\n🎯 Expected Annual Returns:")
    for asset, ret in expected_returns.items():
        print(f"  {asset}: {ret:.2%}")
    
    # Volatilities (standard deviations)
    print("\n📊 Annual Volatilities:")
    volatilities = np.sqrt(np.diag(cov_matrix))
    for asset, vol in zip(expected_returns.index, volatilities):
        print(f"  {asset}: {vol:.2%}")
    
    # Sharpe ratios (individual assets)
    print(f"\n⚡ Individual Sharpe Ratios (Risk-free rate: {RISK_FREE_RATE:.1%}):")
    for asset, ret, vol in zip(expected_returns.index, expected_returns, volatilities):
        sharpe = (ret - RISK_FREE_RATE) / vol
        print(f"  {asset}: {sharpe:.3f}")
    
    print(f"\n🔗 Correlation Matrix:")
    print(correlation_matrix.round(3))

# Calculate portfolio statistics
expected_returns, cov_matrix, correlation_matrix = calculate_portfolio_stats(returns_df)
display_portfolio_stats(expected_returns, cov_matrix, correlation_matrix)

## Step 3: Monte Carlo Portfolio Simulation

We'll generate thousands of random portfolio weights and calculate their risk-return characteristics.

In [ ]:
def generate_random_portfolios(expected_returns, cov_matrix, num_portfolios, risk_free_rate):
    """
    Generate random portfolios using Monte Carlo simulation.
    
    Parameters:
    - expected_returns: Expected returns for each asset
    - cov_matrix: Covariance matrix
    - num_portfolios: Number of random portfolios to generate
    - risk_free_rate: Risk-free rate for Sharpe ratio calculation
    
    Returns:
    - results: Dictionary containing portfolio metrics
    """
    n_assets = len(expected_returns)
    results = {
        'returns': [],
        'volatility': [],
        'sharpe_ratio': [],
        'weights': []
    }
    
    print(f"🎲 Generating {num_portfolios:,} random portfolios...")
    
    for i in range(num_portfolios):
        # Generate random weights that sum to 1
        weights = np.random.random(n_assets)
        weights = weights / np.sum(weights)
        
        # Calculate portfolio return
        portfolio_return = np.sum(expected_returns * weights)
        
        # Calculate portfolio volatility
        portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
        portfolio_volatility = np.sqrt(portfolio_variance)
        
        # Calculate Sharpe ratio
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility
        
        # Store results
        results['returns'].append(portfolio_return)
        results['volatility'].append(portfolio_volatility)
        results['sharpe_ratio'].append(sharpe_ratio)
        results['weights'].append(weights)
        
        # Progress indicator
        if (i + 1) % (num_portfolios // 10) == 0:
            print(f"  Progress: {((i + 1) / num_portfolios) * 100:.0f}%")
    
    # Convert to numpy arrays
    for key in ['returns', 'volatility', 'sharpe_ratio']:
        results[key] = np.array(results[key])
    
    print(f"✅ Generated {num_portfolios:,} portfolios")
    
    return results

# Generate random portfolios
portfolio_results = generate_random_portfolios(
    expected_returns, cov_matrix, NUM_PORTFOLIOS, RISK_FREE_RATE
)

# Display summary statistics
print(f"\n📊 Portfolio Simulation Results:")
print(f"Returns - Min: {portfolio_results['returns'].min():.2%}, Max: {portfolio_results['returns'].max():.2%}")
print(f"Volatility - Min: {portfolio_results['volatility'].min():.2%}, Max: {portfolio_results['volatility'].max():.2%}")
print(f"Sharpe Ratio - Min: {portfolio_results['sharpe_ratio'].min():.3f}, Max: {portfolio_results['sharpe_ratio'].max():.3f}")

## Step 4: Identify Optimal Portfolio

We'll find the portfolio with the maximum Sharpe ratio and display its characteristics.

In [ ]:
def find_optimal_portfolio(portfolio_results, expected_returns, assets):
    """
    Find the portfolio with maximum Sharpe ratio.
    
    Parameters:
    - portfolio_results: Results from Monte Carlo simulation
    - expected_returns: Expected returns for validation
    - assets: List of asset names
    
    Returns:
    - optimal_portfolio: Dictionary with optimal portfolio characteristics
    """
    # Find index of maximum Sharpe ratio
    max_sharpe_idx = np.argmax(portfolio_results['sharpe_ratio'])
    
    # Extract optimal portfolio characteristics
    optimal_portfolio = {
        'weights': portfolio_results['weights'][max_sharpe_idx],
        'return': portfolio_results['returns'][max_sharpe_idx],
        'volatility': portfolio_results['volatility'][max_sharpe_idx],
        'sharpe_ratio': portfolio_results['sharpe_ratio'][max_sharpe_idx]
    }
    
    return optimal_portfolio, max_sharpe_idx

def display_optimal_portfolio(optimal_portfolio, assets):
    """Display optimal portfolio characteristics."""
    print("🏆 Optimal Portfolio (Maximum Sharpe Ratio)")
    print("=" * 50)
    print(f"Expected Return: {optimal_portfolio['return']:.2%}")
    print(f"Volatility: {optimal_portfolio['volatility']:.2%}")
    print(f"Sharpe Ratio: {optimal_portfolio['sharpe_ratio']:.3f}")
    
    print(f"\n💼 Portfolio Weights:")
    weights_df = pd.DataFrame({
        'Asset': assets,
        'Weight': optimal_portfolio['weights'],
        'Weight %': optimal_portfolio['weights'] * 100
    })
    weights_df = weights_df.sort_values('Weight %', ascending=False)
    
    for _, row in weights_df.iterrows():
        print(f"  {row['Asset']}: {row['Weight %']:.1f}%")
    
    return weights_df

# Find and display optimal portfolio
optimal_portfolio, max_sharpe_idx = find_optimal_portfolio(portfolio_results, expected_returns, ASSETS)
weights_df = display_optimal_portfolio(optimal_portfolio, ASSETS)

# Also find minimum volatility portfolio for comparison
min_vol_idx = np.argmin(portfolio_results['volatility'])
min_vol_portfolio = {
    'weights': portfolio_results['weights'][min_vol_idx],
    'return': portfolio_results['returns'][min_vol_idx],
    'volatility': portfolio_results['volatility'][min_vol_idx],
    'sharpe_ratio': portfolio_results['sharpe_ratio'][min_vol_idx]
}

print(f"\n🛡️ Minimum Volatility Portfolio (for comparison):")
print(f"Expected Return: {min_vol_portfolio['return']:.2%}")
print(f"Volatility: {min_vol_portfolio['volatility']:.2%}")
print(f"Sharpe Ratio: {min_vol_portfolio['sharpe_ratio']:.3f}")

## Step 5: Visualize Efficient Frontier

We'll create visualizations showing the efficient frontier and highlight the optimal portfolios.

In [ ]:
def plot_efficient_frontier(portfolio_results, optimal_portfolio, min_vol_portfolio, 
                          max_sharpe_idx, min_vol_idx, assets, expected_returns):
    """
    Plot the efficient frontier with highlighted optimal portfolios.
    
    Parameters:
    - portfolio_results: Monte Carlo simulation results
    - optimal_portfolio: Max Sharpe ratio portfolio
    - min_vol_portfolio: Minimum volatility portfolio
    - max_sharpe_idx: Index of max Sharpe portfolio
    - min_vol_idx: Index of min volatility portfolio
    - assets: List of asset names
    - expected_returns: Individual asset expected returns
    """
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Main Efficient Frontier Plot
    scatter = ax1.scatter(portfolio_results['volatility'], portfolio_results['returns'], 
                         c=portfolio_results['sharpe_ratio'], cmap='viridis', alpha=0.6, s=20)
    
    # Highlight optimal portfolios
    ax1.scatter(optimal_portfolio['volatility'], optimal_portfolio['return'], 
               marker='*', color='red', s=300, label='Max Sharpe Ratio')
    ax1.scatter(min_vol_portfolio['volatility'], min_vol_portfolio['return'], 
               marker='*', color='blue', s=300, label='Min Volatility')
    
    # Plot individual assets
    asset_volatilities = [np.sqrt(cov_matrix.loc[asset, asset]) for asset in assets]
    ax1.scatter(asset_volatilities, expected_returns, marker='D', s=100, 
               color='orange', alpha=0.8, label='Individual Assets')
    
    # Add asset labels
    for i, asset in enumerate(assets):
        ax1.annotate(asset, (asset_volatilities[i], expected_returns[i]), 
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    ax1.set_xlabel('Volatility (Risk)')
    ax1.set_ylabel('Expected Return')
    ax1.set_title('Efficient Frontier - Risk vs Return')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Add colorbar
    plt.colorbar(scatter, ax=ax1, label='Sharpe Ratio')
    
    # 2. Sharpe Ratio Distribution
    ax2.hist(portfolio_results['sharpe_ratio'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    ax2.axvline(optimal_portfolio['sharpe_ratio'], color='red', linestyle='--', linewidth=2, 
               label=f'Max Sharpe: {optimal_portfolio["sharpe_ratio"]:.3f}')
    ax2.set_xlabel('Sharpe Ratio')
    ax2.set_ylabel('Frequency')
    ax2.set_title('Distribution of Sharpe Ratios')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Portfolio Weights Pie Chart (Optimal Portfolio)
    colors = plt.cm.Set3(np.linspace(0, 1, len(assets)))
    wedges, texts, autotexts = ax3.pie(optimal_portfolio['weights'], labels=assets, autopct='%1.1f%%', 
                                      colors=colors, startangle=90)
    ax3.set_title('Optimal Portfolio Weights\n(Max Sharpe Ratio)')
    
    # 4. Risk-Return Scatter (Individual Assets vs Optimal Portfolio)
    ax4.scatter(asset_volatilities, expected_returns, s=150, alpha=0.7, 
               label='Individual Assets', color='orange')
    ax4.scatter(optimal_portfolio['volatility'], optimal_portfolio['return'], 
               marker='*', s=300, color='red', label='Optimal Portfolio')
    
    # Add asset labels
    for i, asset in enumerate(assets):
        ax4.annotate(asset, (asset_volatilities[i], expected_returns[i]), 
                    xytext=(5, 5), textcoords='offset points')
    
    ax4.set_xlabel('Volatility (Risk)')
    ax4.set_ylabel('Expected Return')
    ax4.set_title('Individual Assets vs Optimal Portfolio')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Create the visualization
plot_efficient_frontier(portfolio_results, optimal_portfolio, min_vol_portfolio, 
                       max_sharpe_idx, min_vol_idx, ASSETS, expected_returns)

## Step 6: Calculate VaR and CVaR

We'll implement Monte Carlo simulation to estimate Value at Risk and Conditional Value at Risk for our optimal portfolio.

In [ ]:
def calculate_var_cvar_monte_carlo(weights, returns_df, confidence_level=0.95, 
                                 time_horizon=1, num_simulations=10000):
    """
    Calculate VaR and CVaR using Monte Carlo simulation.
    
    Parameters:
    - weights: Portfolio weights
    - returns_df: Historical returns data
    - confidence_level: Confidence level (e.g., 0.95 for 95%)
    - time_horizon: Time horizon in days
    - num_simulations: Number of Monte Carlo simulations
    
    Returns:
    - var: Value at Risk
    - cvar: Conditional Value at Risk
    - simulated_returns: Array of simulated returns
    """
    print(f"🎯 Calculating VaR and CVaR using Monte Carlo simulation...")
    print(f"  Confidence Level: {confidence_level:.1%}")
    print(f"  Time Horizon: {time_horizon} day(s)")
    print(f"  Simulations: {num_simulations:,}")
    
    # Calculate portfolio returns
    portfolio_returns = (returns_df * weights).sum(axis=1)
    
    # Calculate mean and standard deviation
    mean_return = portfolio_returns.mean()
    std_return = portfolio_returns.std()
    
    # Generate simulated returns
    np.random.seed(SEED)
    simulated_returns = np.random.normal(
        mean_return * time_horizon, 
        std_return * np.sqrt(time_horizon), 
        num_simulations
    )
    
    # Calculate VaR (percentile)
    alpha = 1 - confidence_level
    var = np.percentile(simulated_returns, alpha * 100)
    
    # Calculate CVaR (expected value of returns below VaR)
    cvar = simulated_returns[simulated_returns <= var].mean()
    
    # Also calculate parametric VaR for comparison
    parametric_var = norm.ppf(alpha, mean_return * time_horizon, std_return * np.sqrt(time_horizon))
    
    print(f"✅ Risk Metrics Calculated:")
    print(f"  Monte Carlo VaR ({confidence_level:.0%}): {var:.2%}")
    print(f"  Monte Carlo CVaR ({confidence_level:.0%}): {cvar:.2%}")
    print(f"  Parametric VaR ({confidence_level:.0%}): {parametric_var:.2%}")
    
    return var, cvar, simulated_returns, parametric_var

def plot_var_cvar(simulated_returns, var, cvar, confidence_level):
    """
    Plot the distribution of simulated returns with VaR and CVaR markers.
    
    Parameters:
    - simulated_returns: Array of simulated returns
    - var: Value at Risk
    - cvar: Conditional Value at Risk
    - confidence_level: Confidence level
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # 1. Histogram with VaR and CVaR
    ax1.hist(simulated_returns, bins=100, alpha=0.7, color='lightblue', 
            density=True, edgecolor='black', linewidth=0.5)
    
    # Mark VaR and CVaR
    ax1.axvline(var, color='red', linestyle='--', linewidth=2, 
               label=f'VaR ({confidence_level:.0%}): {var:.2%}')
    ax1.axvline(cvar, color='darkred', linestyle='--', linewidth=2, 
               label=f'CVaR ({confidence_level:.0%}): {cvar:.2%}')
    
    # Shade the tail region
    tail_returns = simulated_returns[simulated_returns <= var]
    ax1.hist(tail_returns, bins=50, alpha=0.8, color='red', density=True, 
            label=f'Tail Risk ({100-confidence_level*100:.0f}%)')
    
    ax1.set_xlabel('Portfolio Returns')
    ax1.set_ylabel('Density')
    ax1.set_title(f'Distribution of Simulated Returns\nVaR vs CVaR at {confidence_level:.0%} Confidence')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Cumulative Distribution
    sorted_returns = np.sort(simulated_returns)
    probabilities = np.arange(1, len(sorted_returns) + 1) / len(sorted_returns)
    
    ax2.plot(sorted_returns, probabilities, color='blue', linewidth=2)
    ax2.axvline(var, color='red', linestyle='--', linewidth=2, 
               label=f'VaR ({confidence_level:.0%})')
    ax2.axhline(1 - confidence_level, color='red', linestyle=':', alpha=0.7)
    
    ax2.set_xlabel('Portfolio Returns')
    ax2.set_ylabel('Cumulative Probability')
    ax2.set_title('Cumulative Distribution of Returns')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Calculate VaR and CVaR for optimal portfolio
var, cvar, simulated_returns, parametric_var = calculate_var_cvar_monte_carlo(
    optimal_portfolio['weights'], returns_df, CONFIDENCE_LEVEL, 
    time_horizon=1, num_simulations=10000 if not SAMPLE_MODE else 5000
)

# Plot the results
plot_var_cvar(simulated_returns, var, cvar, CONFIDENCE_LEVEL)

## Step 7: Portfolio Analysis Summary

Let's create a comprehensive summary of our portfolio analysis results.

In [ ]:
def create_portfolio_summary(optimal_portfolio, var, cvar, assets, weights_df):
    """
    Create a comprehensive summary of portfolio analysis results.
    
    Parameters:
    - optimal_portfolio: Optimal portfolio characteristics
    - var, cvar: Risk metrics
    - assets: List of asset names
    - weights_df: DataFrame with portfolio weights
    
    Returns:
    - summary_df: DataFrame with key portfolio metrics
    """
    print("📋 PORTFOLIO ANALYSIS SUMMARY")
    print("=" * 60)
    
    # Portfolio Performance Metrics
    print(f"\n🎯 OPTIMAL PORTFOLIO PERFORMANCE:")
    print(f"  Expected Annual Return: {optimal_portfolio['return']:.2%}")
    print(f"  Annual Volatility: {optimal_portfolio['volatility']:.2%}")
    print(f"  Sharpe Ratio: {optimal_portfolio['sharpe_ratio']:.3f}")
    print(f"  Risk-Free Rate: {RISK_FREE_RATE:.1%}")
    
    # Risk Metrics
    print(f"\n⚠️ RISK METRICS ({CONFIDENCE_LEVEL:.0%} Confidence):")
    print(f"  Value at Risk (VaR): {var:.2%}")
    print(f"  Conditional VaR (CVaR): {cvar:.2%}")
    print(f"  VaR Interpretation: Maximum expected loss on {100-CONFIDENCE_LEVEL*100:.0f}% of days")
    print(f"  CVaR Interpretation: Average loss when VaR is exceeded")
    
    # Portfolio Composition
    print(f"\n💼 PORTFOLIO COMPOSITION:")
    for _, row in weights_df.head().iterrows():
        print(f"  {row['Asset']}: {row['Weight %']:.1f}%")
    
    # Diversification Analysis
    max_weight = weights_df['Weight %'].max()
    min_weight = weights_df['Weight %'].min()
    weight_std = weights_df['Weight %'].std()
    
    print(f"\n📊 DIVERSIFICATION ANALYSIS:")
    print(f"  Highest allocation: {max_weight:.1f}%")
    print(f"  Lowest allocation: {min_weight:.1f}%")
    print(f"  Weight standard deviation: {weight_std:.1f}%")
    
    if weight_std < 10:
        diversification_level = "High (well diversified)"
    elif weight_std < 20:
        diversification_level = "Medium (moderately diversified)"
    else:
        diversification_level = "Low (concentrated)"
    
    print(f"  Diversification level: {diversification_level}")
    
    # Performance Comparison
    print(f"\n📈 PERFORMANCE COMPARISON:")
    individual_sharpes = []
    for asset in assets:
        asset_return = expected_returns[asset]
        asset_vol = np.sqrt(cov_matrix.loc[asset, asset])
        asset_sharpe = (asset_return - RISK_FREE_RATE) / asset_vol
        individual_sharpes.append(asset_sharpe)
        print(f"  {asset} Sharpe Ratio: {asset_sharpe:.3f}")
    
    best_individual_sharpe = max(individual_sharpes)
    improvement = optimal_portfolio['sharpe_ratio'] - best_individual_sharpe
    print(f"  Best Individual Asset Sharpe: {best_individual_sharpe:.3f}")
    print(f"  Portfolio Improvement: {improvement:.3f} ({improvement/best_individual_sharpe*100:+.1f}%)")
    
    # Create summary DataFrame
    summary_data = {
        'Metric': [
            'Expected Return', 'Volatility', 'Sharpe Ratio', 
            'VaR (95%)', 'CVaR (95%)', 'Max Weight', 'Min Weight'
        ],
        'Value': [
            f"{optimal_portfolio['return']:.2%}",
            f"{optimal_portfolio['volatility']:.2%}",
            f"{optimal_portfolio['sharpe_ratio']:.3f}",
            f"{var:.2%}",
            f"{cvar:.2%}",
            f"{max_weight:.1f}%",
            f"{min_weight:.1f}%"
        ]
    }
    
    summary_df = pd.DataFrame(summary_data)
    
    return summary_df

# Create and display portfolio summary
summary_df = create_portfolio_summary(optimal_portfolio, var, cvar, ASSETS, weights_df)

print(f"\n📊 Summary Table:")
print(summary_df.to_string(index=False))

## Step 8: Discussion Questions and Next Steps

### 🤔 Discussion Questions:

1. **What is the diversification effect observed in this portfolio?**
   - Compare the portfolio's Sharpe ratio to individual asset Sharpe ratios
   - How does correlation between assets affect the diversification benefit?

2. **How does the Sharpe ratio guide portfolio allocation?**
   - Why is the maximum Sharpe ratio portfolio considered "optimal"?
   - What are the limitations of using Sharpe ratio as the sole optimization criterion?

3. **What insights do VaR and CVaR provide for risk management?**
   - How would you use these metrics in practice?
   - What are the differences between Monte Carlo and parametric VaR estimation?

### 🚀 Next Steps and Extensions:

#### TODO Exercise 1: Risk-Free Asset Integration
```python
# TODO: Add a risk-free asset to the portfolio and recompute the Sharpe ratio
# Hint: Create a new asset with return = RISK_FREE_RATE and volatility = 0
# How does this change the efficient frontier?
```

#### TODO Exercise 2: Constrained Optimization
```python
# TODO: Implement portfolio optimization with constraints
# - No short selling (weights >= 0)
# - Maximum weight per asset (e.g., <= 40%)
# - Minimum weight per asset (e.g., >= 5%)
# Use scipy.optimize.minimize with appropriate constraints
```

#### TODO Exercise 3: Analytical Optimization
```python
# TODO: Implement analytical portfolio optimization using scipy.optimize
# Compare results with Monte Carlo approach
# Which method is more efficient? More accurate?
```

### 📚 Key Takeaways:
- **Portfolio diversification** reduces risk without necessarily reducing expected return
- **Monte Carlo simulation** provides intuitive understanding of portfolio behavior
- **Sharpe ratio optimization** balances risk and return effectively
- **VaR and CVaR** quantify tail risk for risk management decisions
- **Efficient frontier** visualizes the risk-return trade-off space

---

**Runtime Summary:** This notebook completed in approximately {runtime} minutes in SAMPLE_MODE.

In [ ]:
# Final execution summary
import time
end_time = time.time()

print("✅ NOTEBOOK EXECUTION COMPLETED")
print("=" * 50)
print(f"📊 Portfolio Analysis Summary:")
print(f"  - Analyzed {len(ASSETS)} assets over {len(returns_df)} trading days")
print(f"  - Generated {NUM_PORTFOLIOS:,} random portfolios")
print(f"  - Optimal Sharpe Ratio: {optimal_portfolio['sharpe_ratio']:.3f}")
print(f"  - Portfolio VaR (95%): {var:.2%}")
print(f"  - Portfolio CVaR (95%): {cvar:.2%}")

print(f"\n🎯 Key Insights:")
print(f"  - Diversification improved Sharpe ratio vs individual assets")
print(f"  - Monte Carlo simulation revealed efficient frontier")
print(f"  - Risk management metrics (VaR/CVaR) quantified tail risk")

print(f"\n⏱️ Execution completed successfully!")
if SAMPLE_MODE:
    print(f"  Note: Running in SAMPLE_MODE for faster execution")
    print(f"  Set SAMPLE_MODE=False for full analysis")